### Exercise 12
Implement Logistic Regression with Mini-batch Gradient Descent using TensorFlow. Train it and evaluate it on the moons dataset (introduced in Chapter 5). Try adding all the bells and whistles:

• Define the graph within a logistic_regression() function that can be reused easily.

• Save checkpoints using a Saver at regular intervals during training, and save the final model at the end of training.

• Restore the last checkpoint upon startup if training was interrupted.

• Define the graph using nice scopes so the graph looks good in TensorBoard.

• Add summaries to visualize the learning curves in TensorBoard.

• Try tweaking some hyperparameters such as the learning rate or the mini- batch size and look at the shape of the learning curve.

In [1]:
% matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# prepare dataset

from sklearn import datasets
iris = datasets.load_iris()

X_train = iris['data'][:, 2:] # patal width + patel length
y_train = (iris['target'] == 2).astype(np.float32) # 1 if Iris-virginica, else 0
y_train = y_train.reshape(150, 1)

In [3]:
X_train_plus = np.c_[np.ones((X_train.shape[0], 1)), X_train]

In [4]:
def plot_cost_data(cost_val):
    plt.plot(range(len(cost_val)), cost_val, 'b-')
    plt.xlabel('n_epochs')
    plt.ylabel('cost')
    plt.show()

In [5]:
import tensorflow as tf

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [6]:
# cost function
def build_logistic_regression_graph(learning_rate):
    with tf.name_scope("Input"):
        X = tf.placeholder(tf.float32, shape=(None, 3), name="X")
        y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
    theta = tf.Variable(tf.random_uniform([3, 1], -1.0, 1.0), name="Theta")
    with tf.name_scope("Cost_Function"):
        sigmoid = tf.sigmoid(tf.matmul(X, theta), name="sigmoid")
        cost_part_1 = tf.multiply(y, tf.log(sigmoid))
        cost_part_2 = tf.multiply(tf.subtract(1.0, y), tf.log(tf.subtract(1.0, sigmoid)))
        cost = tf.negative(tf.reduce_sum(tf.add(cost_part_1, cost_part_2)))
    with tf.name_scope("Train"):
        gradient = tf.gradients(cost, [theta])[0]
        training_op = tf.assign(theta, theta - learning_rate * gradient, name="training_op")
    with tf.name_scope("Init"):
        init = tf.global_variables_initializer()
    with tf.name_scope("Save"):
        saver = tf.train.Saver()
    return X, y, training_op, theta, cost, init, saver

In [ ]:
n_epochs = 1000
learning_rate = 0.001

X, y, training_op, theta, cost, init, saver = build_logistic_regression_graph(0.001)
cost_val = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        sess.run(training_op, feed_dict={X : X_train_plus, y : y_train})
        cost_val.append(cost.eval(feed_dict={X : X_train_plus, y : y_train}))
    best_theta = theta.eval()
print(best_theta)
plot_cost_data(cost_val)

## Save and Restore

In [ ]:
n_epochs = 500
learning_rate = 0.001

X, y, training_op, theta, cost, init, saver = build_logistic_regression_graph(0.001)
cost_val = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        sess.run(training_op, feed_dict={X : X_train_plus, y : y_train})
        cost_val.append(cost.eval(feed_dict={X : X_train_plus, y : y_train}))
        if i % 100 == 0:
            saver.save(sess, '/Users/chance/machine_learning/hands_on_machine_learning/tmp/ex9_12_save.ckpt')
    best_theta = theta.eval()
print(best_theta)
plot_cost_data(cost_val)

In [ ]:
# continue training
n_epochs = 1500
with tf.Session() as sess:
    saver.restore(sess, '/Users/chance/machine_learning/hands_on_machine_learning/tmp/ex9_12_save.ckpt')
    for i in range(n_epochs):
        sess.run(training_op, feed_dict={X : X_train_plus, y : y_train})
        cost_val.append(cost.eval(feed_dict={X : X_train_plus, y : y_train}))
    best_theta = theta.eval()
print(best_theta)
plot_cost_data(cost_val)

## Visualization

In [7]:
from datetime import datetime
def create_log_file_path():
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_dir = "tf_logs"
    logdir = "{}/run-{}".format(root_dir, now)
    return logdir

In [ ]:
n_epochs = 1000
learning_rate = 0.002

X, y, training_op, theta, cost, init, saver = build_logistic_regression_graph(0.001)

log_dir = create_log_file_path()
cost_summary = tf.summary.scalar("Cost", cost)
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        sess.run(training_op, feed_dict={X : X_train_plus, y : y_train})
        summary_str = cost_summary.eval(feed_dict={X : X_train_plus, y : y_train})
        file_writer.add_summary(summary_str, i)
    best_theta = theta.eval()
print(best_theta)
file_writer.flush()
file_writer.close()